In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('train.csv', index_col=0)
df.head()

,Market,Day,Stock,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y,Weight
Index,,,,,,,,,,,,,,,,
0,1,1,363,0.002696,0.530040,0.400810,1.300048e-06,4.639533e-06,1.073594e-05,2.180516e-05,4.416943e-05,0.002212,6.214737e-07,200.000000,-0.000196,1.533780
1,1,1,1223,0.081093,1.366218,1.325157,1.656861e-05,3.024668e-05,5.261872e-05,4.193762e-05,4.193762e-05,0.010932,3.414868e-05,300.000000,-0.003798,0.843604
2,1,1,1372,0.009561,0.464871,0.328008,6.500872e-08,2.323204e-07,5.182686e-07,1.080834e-06,2.030442e-06,0.000306,4.079460e-08,159.692076,-0.000003,29.824024
3,1,1,1942,0.045300,0.608222,0.456662,1.928694e-08,6.495275e-08,1.338509e-07,2.644214e-07,6.980798e-07,0.000205,9.502066e-09,212.160378,-0.000008,100.388668
4,1,1,2712,0.001605,0.476718,0.205069,6.759866e-07,2.638783e-06,6.424519e-06,1.189923e-05,2.064450e-05,0.001525,3.428403e-07,153.724351,0.000065,5.950714


In [4]:
del df['Market']
del df['Day']
del df['Stock']

In [5]:
del df['Weight']

In [6]:
df.fillna(0)
df.head()

,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y
Index,,,,,,,,,,,,
0,0.002696,0.530040,0.400810,1.300048e-06,4.639533e-06,1.073594e-05,2.180516e-05,4.416943e-05,0.002212,6.214737e-07,200.000000,-0.000196
1,0.081093,1.366218,1.325157,1.656861e-05,3.024668e-05,5.261872e-05,4.193762e-05,4.193762e-05,0.010932,3.414868e-05,300.000000,-0.003798
2,0.009561,0.464871,0.328008,6.500872e-08,2.323204e-07,5.182686e-07,1.080834e-06,2.030442e-06,0.000306,4.079460e-08,159.692076,-0.000003
3,0.045300,0.608222,0.456662,1.928694e-08,6.495275e-08,1.338509e-07,2.644214e-07,6.980798e-07,0.000205,9.502066e-09,212.160378,-0.000008
4,0.001605,0.476718,0.205069,6.759866e-07,2.638783e-06,6.424519e-06,1.189923e-05,2.064450e-05,0.001525,3.428403e-07,153.724351,0.000065


In [7]:
for column in df.columns:
    df.fillna(0, inplace=True)
    print(np.where(np.isnan(df[column])))

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


In [8]:
X, y = df.loc[:, 'x0':'x6'], df.loc[:,'y']

In [9]:
X, y = X.values, y.values

In [10]:
print(np.where(np.isnan(X)))
print(np.where(y > 1))
print(np.where(y < -1))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64),)
(array([], dtype=int64),)


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(X)
print(scaler.data_max_)
X_std = scaler.transform(X)

[4.99827504e+06 2.94626390e+03 6.77912473e+02 2.83150876e-02
 6.51416194e-02 1.09440456e-01 2.06560350e-01 3.61169254e-01
 1.02200000e-01 3.58925753e-02 7.34914405e+05]


In [12]:
print(X.shape)
print(X_std.shape)
X_train = X_std.reshape(X_std.shape[0], 1, X_std.shape[1])

(623817, 11)
(623817, 11)


In [13]:
from keras.models import Sequential

/home/lambda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
from keras.layers import Dense, Activation, LSTM, Dropout, LeakyReLU

In [17]:
from keras.models import load_model
# run this to load
model = load_model('more_more_lstm.hd5')

In [20]:
# model = Sequential()
batch_size = 32

In [21]:
_ = X_train.shape[0] // batch_size * batch_size
X_train = X_train[:_, :, :]
y_train = y[:_]
print(X_train.shape)
print(y_train.shape)

(623808, 1, 11)
(623808,)


In [33]:
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))

In [34]:
model.add(Dense(1))
model.add(Activation('tanh'))

In [35]:
model.compile(loss='mse', optimizer='adam')

In [22]:
model.fit(X_train, y_train, epochs=40, batch_size=256, shuffle=True, validation_split=0.15)

Train on 530236 samples, validate on 93572 samples
Epoch 1/40
530236/530236 [==============================] - 13s 25us/step - loss: 1.0745e-06 - val_loss: 1.1553e-06
Epoch 2/40
530236/530236 [==============================] - 13s 25us/step - loss: 1.0748e-06 - val_loss: 1.2054e-06
Epoch 3/40
530236/530236 [==============================] - 14s 26us/step - loss: 1.0762e-06 - val_loss: 1.1061e-06
Epoch 4/40
530236/530236 [==============================] - 14s 26us/step - loss: 1.0757e-06 - val_loss: 1.1218e-06
Epoch 5/40
530236/530236 [==============================] - 13s 24us/step - loss: 1.0765e-06 - val_loss: 1.1463e-06
Epoch 6/40
530236/530236 [==============================] - 13s 25us/step - loss: 1.0754e-06 - val_loss: 1.1488e-06
Epoch 7/40
530236/530236 [==============================] - 14s 27us/step - loss: 1.0748e-06 - val_loss: 1.1202e-06
Epoch 8/40
530236/530236 [==============================] - 14s 26us/step - loss: 1.0764e-06 - val_loss: 1.1054e-06
Epoch 9/40
530236/530

In [37]:
model.save('more_more_lstm.hd5')

In [23]:
model.evaluate(x=X_train, y=y_train, batch_size=batch_size)

623808/623808 [==============================] - 20s 32us/step


1.0663897078412986e-06

In [24]:
yhat = model.predict(X_train[:1000], batch_size=32)
yhat[:3]

array([[-2.0864527e-05],
       [ 7.2607351e-04],
       [-3.3096421e-05]], dtype=float32)

In [25]:
expected = list(y[:1000])

In [26]:
pred = [l[0] for l in yhat]

In [27]:
diff = np.asarray(pred) - np.asarray(expected)

In [28]:
diff.shape

(1000,)

In [29]:
err = diff / np.asarray(expected)

In [30]:
err = err * 100

In [31]:
buckets = [1, 5, 10, 25, 50, 100, 10000]
for bucket in buckets:
    print("%d percent of data has err < %d" % (np.where(err < bucket)[0].size, bucket))

945 percent of data has err < 1
947 percent of data has err < 5
948 percent of data has err < 10
957 percent of data has err < 25
960 percent of data has err < 50
967 percent of data has err < 100
1000 percent of data has err < 10000


In [ ]:
tdf = pd.read_csv('test.csv', index_col=0)
tdf.head()

In [48]:
del tdf['Market']
del tdf['Day']
del tdf['Stock']

In [49]:
for column in tdf.columns:
    tdf.fillna(0, inplace=True)
    print(np.where(np.isnan(tdf[column])))
tdf.head()

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6
Index,,,,,,,,,,,
0,0.000842,0.436594,0.477848,1.535381e-06,4.353950e-06,7.718616e-06,1.632886e-05,3.267851e-05,0.001661,9.724611e-07,112.968475
1,0.020800,0.636143,0.670798,5.204133e-06,9.546012e-06,1.575258e-05,2.804666e-05,1.082500e-05,0.002104,2.146299e-05,100.000000
2,0.068580,0.478891,0.481816,5.433730e-08,1.733539e-07,3.866963e-07,8.520664e-07,2.103176e-06,0.000222,3.085296e-08,188.335508
3,0.011986,0.463201,0.294900,1.103215e-08,3.555558e-08,7.280376e-08,1.364231e-07,3.155050e-07,0.000141,1.678216e-08,212.123913
4,0.000164,0.549980,0.352332,4.494670e-07,8.503405e-07,1.222847e-06,1.901114e-06,4.048304e-06,0.001235,4.595200e-07,124.024977


In [50]:
X_test = tdf.values
X_test = scaler.transform(X_test)
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_test.shape)
print(X_test[0])
print(X_test[1])

(640430, 1, 11)
[[-1.         -0.99970363 -0.99859024 -0.99989155 -0.99986632 -0.99985894
  -0.9998419  -0.99981904 -0.96758707 -0.99994581 -0.99969529]]
[[-0.99999999 -0.99956817 -0.99802099 -0.99963241 -0.99970692 -0.99971213
  -0.99972844 -0.99994006 -0.95890689 -0.99880404 -0.99973058]]


In [ ]:
yhat = model.predict(X_test, batch_size=32)

In [ ]:
print(yhat.shape)
yp = pd.Series(yhat.reshape(yhat.shape[0])).rename('y')

In [ ]:
yp.index.name = 'Index'
yp.head()

In [ ]:
yp.to_csv('more_more_lstm.csv', header=True)

In [144]:
model.save('more_lstm.hd5')